<a href="https://colab.research.google.com/github/Tristkick1234/Signify/blob/main/signify3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d datamunge/sign-language-mnist

#dont forget to download your api key and place it in Files


 72% 45.0M/62.6M [00:00<00:00, 124MB/s] 
100% 62.6M/62.6M [00:00<00:00, 127MB/s]


In [4]:
!unzip sign-language-mnist.zip

Archive:  sign-language-mnist.zip
  inflating: amer_sign2.png          
  inflating: amer_sign3.png          
  inflating: american_sign_language.PNG  
  inflating: sign_mnist_test.csv     
  inflating: sign_mnist_test/sign_mnist_test.csv  
  inflating: sign_mnist_train.csv    
  inflating: sign_mnist_train/sign_mnist_train.csv  


In [5]:
#Importing libraries

import cv2
import pandas as pd
import numpy as np


#all the different layers needed
import keras
from keras.utils import to_categorical
from keras.layers.convolutional import Conv2D
from keras.layers import Dense
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Dropout



##Preprocessing: transforming the data so the data can be used by the model

In [6]:
train = pd.read_csv("sign_mnist_train/sign_mnist_train.csv")
test = pd.read_csv("sign_mnist_test/sign_mnist_test.csv")

x_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
x_test = test.iloc[:,1:]
y_test = test.iloc[:,0]

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train = x_train.values.reshape(-1,28,28,1)
x_test = x_test.values.reshape(-1,28,28,1)


In [ ]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


##Building the model

In [7]:
learning_rate=keras.callbacks.ReduceLROnPlateau(monitor='accuracy',factor=0.1,patience=2,min_lr=0.0001) #loss used for monitoring

In [8]:
#sequential
model = keras.models.Sequential()

#neural network layers
model.add(Conv2D(75, (3,3), strides = 1 , padding = 'same', activation = 'relu', input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides = 2, padding = 'same'))
model.add(Conv2D(50, (3,3), strides = 1 , padding = 'same', activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(MaxPooling2D((2,2), strides = 2, padding = 'same'))
model.add(Conv2D(25 , (3,3), strides=1 , padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides = 2, padding = 'same'))

model.add(Flatten()) #becomes simple vector output
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=25,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 75)        750       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 75)       300       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 75)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout (Dropout)           (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 50)       2

In [ ]:
#Using OpenCv to show result

In [9]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

while True:
    ret, frame = cap.read()

    # Check if the frame was successfully read
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    # Get the height and width of the frame
    height, width, _ = frame.shape

    # Define the region of interest (roi) within the frame
    roi_x = width // 2 - 150
    roi_y = height // 2 - 150
    roi_width = 300
    roi_height = 300
    roi = frame[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]

    # Draw a rectangle around the region of interest
    cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 255, 0), 2)

    # Resize the region of interest to (28, 28)
    resized = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)

    # Convert the resized image to grayscale
    resized_gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    # Make prediction using your model
    pred = model.predict_classes(np.expand_dims(np.expand_dims(resized_gray, axis=2), axis=0))[0]

    # Put the predicted letter on the frame
    cv2.putText(frame, letters[pred], (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 122,), 3)

    # Show the frame and the resized image
    cv2.imshow('frame', frame)
    cv2.imshow('mask', resized_gray)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
